In [39]:
import json
import os
import numpy as np
import pandas as pd
import emoji

root = "verified"

def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False

identifier = []
photos = []
captions = []
collections = 0
no_image = 0
no_caption = 0
has_quotes = 0
scraper_failed = 0
posts_per_user = []
mentions = 0
hashtags = 0
emojis = 0
vanilla = 0
for fn in os.listdir("profiles_" + root + "/"):
    if fn.endswith(".json"):
        f = open("profiles_" + root + "/" + fn)
        num_valid_entries = 0
        num_invalid_entries = 0
        data = json.load(f)
        try:
            posts = data["posts"]
        except:
            if data["num_of_posts"] >= 0 and data["num_of_posts"] <= 100:
                print(data["num_of_posts"])
                scraper_failed += data["num_of_posts"]
            else:
                scraper_failed += 100
        for p in posts:
            if len(p["imgs"]) > 1:
                collections += 1
                num_invalid_entries += 1
            elif len(p["imgs"]) == 0:
                no_image += 1
                num_valid_entries += 1
            elif p["caption"] == "":
                no_caption += 1
                num_invalid_entries += 1
            else:
                num_valid_entries += 1
                vanilla_flag = True
                identifier.append(data["username"])
                photos.append(p["imgs"][0])
                if '"' in p["caption"]:
                    has_quotes += 1
                    vanilla_flag = False
                if '@' in p["caption"]:
                    mentions += 1
                    vanilla_flag = False
                if '#' in p["caption"]:
                    hashtags += 1
                    vanilla_flag = False
                if text_has_emoji(p["caption"]):
                    emojis += 1
                    vanilla_flag = False
                if vanilla_flag:
                    vanilla += 1
                captions.append(p["caption"].replace('"', '\''))
        posts_per_user.append(num_valid_entries)
        if data
        scraper_failed += 100 - num_valid_entries - num_invalid_entries
        f.close()

total = len(captions) + collections + no_image + no_caption
print("scraper failed", scraper_failed, round(scraper_failed * 100/ (total + scraper_failed), 2))
print("throwing out:", collections + no_image + no_caption, round((collections + no_image + no_caption) * 100 / total, 2))
print("\t no caption", no_caption, round((no_caption / total) * 100, 2))
print("\t no image", no_image, round((no_image / total) * 100, 2))
print("\t collection", collections, round((collections / total) * 100, 2))
print("usable:", len(captions), round(len(captions)*100/total, 2))
print("\t mentions", mentions, round((mentions / total) * 100, 2))
print("\t hashtags", hashtags, round((hashtags / total) * 100, 2))
print("\t emojis", emojis, round((emojis / total) * 100, 2))
print("\t has quotes", has_quotes, round((has_quotes / total) * 100, 2))
print("\t vanilla", vanilla, round((vanilla / total) * 100, 2))
print("post per user:", posts_per_user)

with open('data_' + root + '.csv', 'w') as f:
    f.write("username,photo,caption\n")
    for i in range(len(photos)):
        f.write("%s,%s,\"%s\"\n" % (identifier[i],photos[i], captions[i]))
f.close()

try:
    csv = pd.read_csv('data_' + root + '.csv', header=0)
    csv.drop_duplicates(keep="last",inplace=True)
    csv.to_csv('data_' + root + '.csv', index=False)
except:
    print('pandas stuff did not work')


scraper failed 220139 96.94
throwing out: 2845 40.93
	 no caption 404 5.81
	 no image 1533 22.05
	 collection 908 13.06
usable: 4106 59.07
	 mentions 1783 25.65
	 hashtags 1852 26.64
	 emojis 1494 21.49
	 has quotes 135 1.94
	 vanilla 1022 14.7
post per user: [61, 69, 87, 24, 56, 93, 88, 60, 94, 96, 52, 19, 69, 0, 70, 83, 0, 88, 8, 36, 60, 100, 7, 12, 78, 68, 96, 71, 90, 72, 1, 8, 12, 90, 93, 97, 70, 86, 5, 7, 67, 88, 86, 9, 92, 80, 82, 78, 91, 96, 57, 12, 70, 85, 75, 97, 10, 80, 33, 83, 70, 0, 88, 99, 80, 83, 80, 60, 17, 11, 56, 50, 96, 18, 0, 11, 72, 11, 20, 90, 82, 82, 85, 79, 93, 100, 95, 75, 92, 93, 10, 94]
